In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.datasets import imdb
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


## Reading the dataset

In [2]:
from pandas import read_csv
df = pd.read_csv('sample.csv',header=0, encoding='latin-1', sep = ',', names = ['label', 'content'], error_bad_lines=False)

In [3]:
df.head(5)

,label,content
0,Bachelors Degree,A
1,Bachelors Degree,B
2,Bachelors Degree,S
3,Bachelors Degree,BS
4,Bachelors Degree,BT


In [4]:
x1= df['content']
x1.head(5)

0     A
1     B
2     S
3    BS
4    BT
Name: content, dtype: object

In [25]:
vectorizer_CountVectorizer = CountVectorizer(lowercase = False, min_df=0, token_pattern=(r'\w+'))
vectorizer_CountVectorizer.fit(x1)
vectorizer_CountVectorizer.vocabulary_

{'A': 341,
 'B': 1816,
 'S': 10642,
 'D': 3946,
 'M': 7436,
 '3': 188,
 'C': 2620,
 'N': 8508,
 'AA': 353,
 'AE': 982,
 'AS': 1514,
 'GS': 5905,
 'SX': 11550,
 'UD': 12162,
 'AB': 550,
 'AC': 566,
 'BA': 1821,
 'BE': 2043,
 'BF': 2079,
 'BL': 2231,
 'BM': 2246,
 'BS': 2337,
 'BT': 2529,
 'BW': 2616,
 'CS': 3820,
 'DM': 4333,
 'DP': 4397,
 'FA': 5320,
 'OL': 8905,
 'PE': 9249,
 'SB': 10679,
 'UG': 12163,
 'BC': 2023,
 'DC': 3989,
 'DN': 4343,
 'DO': 4350,
 'HD': 5974,
 'JD': 6961,
 'MD': 7818,
 'OD': 8858,
 'PB': 9229,
 'PH': 9338,
 'AM': 1188,
 'CN': 3251,
 'DS': 4454,
 'ND': 8570,
 'NP': 8696,
 'PD': 9245,
 'PT': 10084,
 'SM': 11073,
 'AD': 719,
 'CI': 3108,
 'CJ': 3153,
 'ED': 4584,
 'EE': 4658,
 'EL': 4696,
 'HA': 5934,
 'MA': 7441,
 'MB': 7766,
 'MC': 7784,
 'ME': 7832,
 'MF': 7997,
 'MM': 8178,
 'MR': 8297,
 'MS': 8316,
 'MT': 8435,
 'PP': 9654,
 'PR': 9665,
 'SN': 11084,
 'SP': 11168,
 'SW': 11542,
 'CG': 3001,
 'GC': 5659,
 'GR': 5833,
 'IA': 6324,
 'IB': 6331,
 'PC': 9239,
 'AF

In [ ]:
x1_CountVectorizer_array=vectorizer_CountVectorizer.transform(x1).toarray()
np.set_printoptions(threshold=np.inf)
print(x1_CountVectorizer_array.shape)
print(x1_CountVectorizer_array)   ##(60, 65)

(97409, 12644)


In [ ]:
len(vectorizer_CountVectorizer.vocabulary_)

In [ ]:
x2= df['content']
x2 = x2.apply(str)
x2.head(5)


In [ ]:
x2.shape

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
tokenizer_sequences = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)
## tokenize it to words
tokenizer_sequences.fit_on_texts(x2)
## change each word to a value
sequences = tokenizer_sequences.texts_to_sequences(x2)
print("The sequences generated from text are : ",sequences)


In [ ]:
len(sequences)

In [ ]:
x2_tokenizer_sequences = tokenizer_sequences.sequences_to_matrix(sequences, mode='binary')
print(x2_tokenizer_sequences)

In [ ]:
x2_tokenizer_sequences.shape

In [ ]:
x4= df['content']

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)
## tokenize it to words
tokenizer.fit_on_texts(x4)

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
x4_sequences = tokenizer.texts_to_sequences(x4)

# Get max training sequence length
maxlen = max([len(x) for x in x4_sequences])

# Pad the training sequences
x4_pad_sequences = pad_sequences(x4_sequences, padding="post", truncating='post', maxlen=maxlen)

# Output the results of our work
print("Word x2:\n", x2.head(5))
print("Word index:\n", word_index)
print("\nTraining sequences:\n", x4_sequences)
print("\nmaxlen:\n", maxlen)
print("\nPadded training sequences:\n", x4_pad_sequences)
print("\nPadded training shape:", x4_pad_sequences.shape)
print("Training sequences data type:", type(x4_sequences))
print("Padded Training sequences data type:", type(x4_pad_sequences))

In [ ]:
# we now convert the sentiments into numeric for the machine to understand

from sklearn.preprocessing import LabelEncoder

# now call the class encoder and assign it to a variable, then finally transform

le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])
df.head(5)

In [ ]:
df = df[df.label.apply(lambda x: x !="nan")] 
df = df.reset_index(drop=True)
df['label'].value_counts().plot(kind='bar')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x1_CountVectorizer_array, df['label'], test_size = 0.1, random_state = 5, stratify = df['label'])
x_train

In [ ]:
train_input = x_train.reshape((x_train.shape[0], x_train.shape[1],1))
train_input.shape
text_input = x_test.reshape((x_test.shape[0], x_test.shape[1],1))
text_input.shape

In [ ]:
model = Sequential()
model.add(LSTM(units=20,activation='relu', input_shape=(train_input.shape[1], train_input.shape[2]), return_sequences=True, go_backwards=True))
model.add(LSTM(units=20,activation='relu', return_sequences=False, go_backwards=True))
model.add(Dense(88, activation='relu'))
model.add(Dropout(rate=.3))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
print(model.summary())

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image(filename = "model.png")

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_input, y_train,  epochs=20, validation_split=0.2)

In [ ]:
loss, accuracy = model.evaluate(train_input, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(text_input, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
model = Sequential()
model.add(Embedding(56, 100, input_length=3)) 
### an embedding layer to expand tokens, allowing NN to represent the word in a meaningful way ###
######Embedding(vocabolary size, dimension of embedding, MAX length of each sentence)####
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # 
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])